In [10]:
from __future__ import print_function

import os
import cv2
import numpy as np
import tensorflow.keras
from sklearn import preprocessing

In [17]:
# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return f"{h}:{m:>02}:{s:>05.2f}"

In [2]:
def preprocess(image, imageWidth, imageHeight) :
    return cv2.resize(image, (imageWidth, imageHeight))

In [3]:
def loadData(path, augment=False, preprocess=None):
    x = None
    y = np.empty(shape=(0,))
    subDirs = [name for name in os.listdir(
        path) if os.path.isdir(os.path.join(path, name))]

    for classNumber, subDir in enumerate(subDirs):
        tempX = []
        tempY = []
        classDirPath = os.path.join(path, subDir)
        imageDir = [name for name in os.listdir(
            classDirPath) if os.path.isfile(os.path.join(classDirPath, name))]
        for index, imageName in enumerate(imageDir):
            image = cv2.imread(os.path.join(classDirPath, imageName))
            if (preprocess):
                image = preprocess[0](image, preprocess[1], preprocess[2])

            tempX.insert(index, image)
            tempY.insert(index, subDir)
        if x is None:
            x = np.array(tempX)
        else:
            x = np.append(x, np.array(tempX), axis=0)
        y = np.append(y, np.array(tempY), axis=0)
    return x, y

### test

In [14]:
SUBTRACT_PIXEL_MEAN = True
NUM_CLASSES = 7

In [11]:
valPath = os.path.join("./seti-data/primary_small/valid")
testPath = os.path.join("./seti-data/primary_small/test")

IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224

(xVal, yValStr) = loadData(valPath, preprocess = [preprocess, IMAGE_WIDTH,  IMAGE_HEIGHT])
(xTest, yTestStr) = loadData(testPath, preprocess = [preprocess, IMAGE_WIDTH,  IMAGE_HEIGHT])

In [12]:
le = preprocessing.LabelEncoder()
yVal = le.fit_transform(yValStr)
yTest = le.transform(yTestStr)

In [15]:
xVal = xVal.astype('float32') / 255
xTest = xTest.astype('float32') / 255

if SUBTRACT_PIXEL_MEAN:
    xTrainNMean = np.load('geekfile.npy')
    xVal -= xTrainNMean
    xTest -= xTrainNMean
    
yVal = tensorflow.keras.utils.to_categorical(yVal, NUM_CLASSES)
yTest = tensorflow.keras.utils.to_categorical(yTest, NUM_CLASSES)

In [18]:
import time

start_time = time.time()
new_model = tensorflow.keras.models.load_model("./model/my_model_89.h5")
elapsed_time = time.time() - start_time
print("Elapsed time: {}".format(hms_string(elapsed_time)))

Elapsed time: 0:00:08.29


In [20]:
scores = new_model.evaluate(xVal, yVal, verbose=0)
print(scores)

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node model/conv2d/Conv2D (defined at C:\Users\DRTeam03\Anaconda3\envs\bagus\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]] [Op:__inference_distributed_function_23227]

Function call stack:
distributed_function


In [ ]:
scores = new_model.evaluate(xTest, yTest, verbose=0)
print(scores)